In [ ]:
import pandas as pd
import numpy as np
from sklearn import svm
import nltk, string, pickle
from collections import Counter
from gensim.models import KeyedVectors
from sklearn.model_selection import KFold # import KFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

exclude = set(string.punctuation)

# this function convert a string of text to a cross-lingual doc embedding capturing its semantics
def text_embedding(text,model):
    
    text = text.lower()
    
    text = nltk.word_tokenize(text)
        
    text = [token for token in text if token not in exclude and token.isalpha()]
    
    doc_embedd = []
    
    for word in text:
            try:
                embed_word = model[word]
                doc_embedd.append(embed_word)
            except KeyError:
                continue
    if len(doc_embedd)>1:
        avg = [float(sum(col))/len(col) for col in zip(*doc_embedd)]
        return avg
    else:
        return None

In [ ]:
# we load the dataset

with open('dataset.pickle', 'rb') as f:
    df = pickle.load(f)    

In [ ]:
df.head()

In [ ]:
# for each language under study you need to download its related cross-lingual embeddings from here: https://github.com/facebookresearch/MUSEœ

de_model = KeyedVectors.load_word2vec_format('word-embs/wiki.multi.de.vec')
fr_model = KeyedVectors.load_word2vec_format('word-embs/wiki.multi.fr.vec')
en_model = KeyedVectors.load_word2vec_format('word-embs/wiki.multi.en.vec')
it_model = KeyedVectors.load_word2vec_format('word-embs/wiki.multi.it.vec')
fi_model = KeyedVectors.load_word2vec_format('word-embs/wiki.multi.fi.vec')
pl_model = KeyedVectors.load_word2vec_format('word-embs/wiki.multi.pl.vec')
sl_model = KeyedVectors.load_word2vec_format('word-embs/wiki.multi.sl.vec')

In [ ]:
# we just map the language with the word embeddings model

model_dict = {"fr":fr_model,"en":en_model,"de":de_model,"it":it_model,"fi":fi_model,"pl":pl_model,"sl":sl_model}

In [ ]:
# in case we want to skip a specific topic
skip = [""]
#skip = ["germanDemocraticRepublic.json"]

In [ ]:
# a quick overview of the frequency of language

langs = []
check = []

for index, row in df.iterrows():
    if row["langMaterial"] not in model_dict:
        check.append(row["langMaterial"])
    else:
        langs.append(row["langMaterial"])
        

print(Counter(langs).most_common())

In [ ]:
# for each document we create a document embedding and collect its topic label

embs = []
labels = []
selected_langs = []

for index, row in df.iterrows():
    lang = row["langMaterial"]
    label = row["filename"]
    if lang in model_dict and label not in skip:
        model = model_dict[lang]
        text = row["unitTitle"] +" "+ row["titleProper"]+" "+ row["scopeContent"]
        emb = text_embedding(text,model)
        if emb != None:
            embs.append(emb)
            labels.append(label)
            selected_langs.append(lang)
print (len(embs),len(labels))

In [ ]:
# overview of relation between label and language

check = []
for x in range(len(labels)):
    lang = selected_langs[x]
    label = labels[x]
    check.append(label+" "+lang)
Counter(check).most_common()

In [ ]:
X = np.array(embs)
y = np.array(labels)

In [ ]:
# we evaluate the quality of the classifier via 10fold cross validation
# we report for each fold precision recall f1 and micro f1

SVM = svm.SVC(kernel = "linear", C=1,probability=True)

kf = KFold(n_splits=10,shuffle=True)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    classifier = SVM.fit(X_train , y_train)
    y_pred = classifier.predict(X_test)
    
    p,r,f1,s = precision_recall_fscore_support(y_test, y_pred, average="macro")
    micro_f1 = precision_recall_fscore_support(y_test, y_pred, average="micro")[0]
    print ("p",round(p,2),"r",round(r,2),"f1",round(f1,2),"micro_f1",round(micro_f1,2))
    print (Counter(y_test).most_common())
    print (" ")

In [ ]:
# we train a final model

classifier = SVM.fit(X , y)

# save the model to disk
filename = 'trained_topic_classifier.model'
pickle.dump(classifier, open(filename, 'wb'))

In [ ]:
#load the pretrained model

with open('trained_topic_classifier.model', 'rb') as f:
    classifier = pickle.load(f)

# get its label
cl_labels = classifier.classes_

In [ ]:
# apply the classifier to a new description

description = "this is a text about the GDR and Berlin"
lang = "en"

model = model_dict[lang]
emb = text_embedding(description,model)

In [ ]:
pred_proba = classifier.predict_proba([emb])[0]
preds = [[cl_labels[x],pred_proba[x]] for x in range(len(pred_proba))]
preds.sort(key=lambda x: x[1],reverse=True)

# the probability of each topic
for x in preds:
    print (x[0],round(x[1],3))